<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/stackExchange_Final_NN_Multilabel_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Defined the autoreload functionality
%load_ext autoreload
%autoreload 2

In [2]:
#Imported/installed required packages and defined the google drive access, basepath & added system path to custom-functions
import sys
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')

  !pip install torchtext --upgrade --q
  !pip install torchmetrics --q
  !pip install --quiet torch-lr-finder --q
  !pip install --upgrade wandb --q
  !pip install --gensim --q

  basepath = '/content/drive/My Drive/NLP'
  sys.path.append('/content/drive/My Drive/NLP/custom-functions')

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --gensim


In [3]:
#Importing the required packages
import torch
import torch.nn as nn
import torch.nn.functional as F

import torchmetrics
from torchmetrics.classification import MultilabelF1Score, MultilabelHammingDistance
from torchmetrics.functional.classification import multilabel_f1_score,multilabel_hamming_distance

import joblib
import ast
import wandb

from types import SimpleNamespace
from functools import partial
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer as mlb

In [4]:
#Defined the google drive folders for accessing/saving model progress
embeddings_folder = Path(basepath)/ 'assignment7/WordEmbeddings'
data_folder = Path(basepath)/ 'assignment7/MultiLabel_Classification'
model_saving_folder = Path(basepath)/ 'assignment7/NN_MultiLabel_Classification_task3b'


In [5]:
#Access the joblib file and load the dataset
data_cleaned = data_folder/ "df_multilabel_hw_cleaned.joblib"
stackExchange_dataset = joblib.load(data_cleaned)

In [6]:
#Defined the X & y variables from the dataset
X = stackExchange_dataset['cleaned_text'].values
y = stackExchange_dataset['Tag_Number'].values

In [7]:
#Installed swifter packages
!pip install swifter -qq  

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 830.9/830.9 kB 50.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 85.1 MB/s eta 0:00:00


In [8]:
#Installed required packages, and converted Tag_Number using ast module
import swifter
import ast
stackExchange_dataset['Tag_number_list'] = stackExchange_dataset['Tag_Number'].swifter.apply(lambda x: ast.literal_eval(x))

Pandas Apply:   0%|          | 0/47427 [00:00<?, ?it/s]

In [9]:
y_final = stackExchange_dataset['Tag_number_list'].values

In [10]:
#from sklearn.preprocessing import MultiLabelBinarizer as mlb
y_stackExchange_encoding = mlb().fit_transform(y_final)

In [11]:
#Splitted the dataset into train, test & valid proportionally
from sklearn.model_selection import train_test_split
X_sExchange_train, X_valid_test, y_sExchange_train, y_valid_test = train_test_split(X, y_stackExchange_encoding, test_size=0.4, random_state=42)
X_sExchange_valid, X_sExchange_test, y_sExchange_valid, y_sExchange_test = train_test_split(X_valid_test, y_valid_test, test_size=0.5, random_state=42)

In [12]:
#Accessiing binary file from the given folder and loaded the vectors using KeyedVectors
from gensim.models import KeyedVectors
pretrained_sExchange_file = str(embeddings_folder/ "model_stackExchange_CBOW.bin")
sExchange_vectors = KeyedVectors.load(pretrained_sExchange_file)

In [13]:
#Installed learning rate finder package
!pip install torch_lr_finder -qq

In [14]:
#Importing the Learing Rate finder & other custom packages
from torch_lr_finder import LRFinder
from Trainer_v4 import Trainer
from data_preparation_HW7 import *

In [15]:
#Importing the custom packages
from ff_sequential_model_v1 import MLPCustom

#Creating Dataset & Vocab

In [16]:
#Created the respective custom Dataset 
se_trainset = CustomDataset(X_sExchange_train, y_sExchange_train)
se_validset = CustomDataset(X_sExchange_valid, y_sExchange_valid)
se_testset = CustomDataset(X_sExchange_test, y_sExchange_test)
se_train_subset_indices = random.sample(range(0, len(se_trainset)), len(se_trainset)*0.25)
se_train_subset = torch.utils.data.Subset(se_trainset, se_train_subset_indices)
#Creating stackexchange vocab
stackExchange_vocab = get_vocab(se_trainset, min_freq=2)

In [17]:
#Getting pretrained weights from the generated vocabulary & vectors and the given embedding dimension
pretrained_weights, words_found, words_not_found = get_pretrained_weights(
    vocab=stackExchange_vocab,
    pretrained_vectors=sExchange_vectors,
    embedding_dim = 300,
    )

In [18]:
pretrained_weights.shape

torch.Size([90287, 300])

In [19]:
type(pretrained_weights), words_found, words_not_found

(torch.Tensor, 14664, 75623)

In [20]:
#Defined the hyperparameters needed for the model 
hyperparameters = SimpleNamespace(
    # for model
    EMBED_DIM=300,
    VOCAB_SIZE=len(stackExchange_vocab),
    OUTPUT_DIM=10,
    HIDDEN_SIZES_LIST=[200],
    DPROB_LIST=[0.0],
    NON_LINEARITY=nn.ReLU(),
    BATCH_NORM=False,

    # for training
    INITIALIZATION ='kaiming',
    EPOCHS=20,
    BATCH_SIZE=128,
    LEARNING_RATE=0.001,
    DATASET='IMDB',
    ARCHITECTURE='embed_layer-ffn',

    # for optimizer
    OPTIMIZER='AdamW',
    MOMENTUM = 0,
    NESTEROV = False,
    WEIGHT_DECAY = 0.000,

    # gradient clipping
    CLIP_TYPE='norm',
    CLIP_VALUE=2,

    # early stopping
    EARLY_STOP_PATIENCE=5,

    #scheduler
    SCHEDULER = 'None'
    )

Specify the run name and folder

In [21]:
#Defined the path for saving the project & subsequent model run results
project_name = 'Regularization_stackExchange_v4'
run_name = 'exp1.3'
run_folder = model_saving_folder/project_name/run_name
run_folder.mkdir(exist_ok=True, parents=True)
log_frequency = 5

In [22]:
run_folder

PosixPath('/content/drive/My Drive/NLP/assignment7/NN_MultiLabel_Classification_task3b/Regularization_stackExchange_v4/exp1.2')

In [23]:
###   Given the parameters and its subsequent runs   

# run 1 - based on default initialization
# Notes: Final Learning rate was set to 0.01

# run 2 - add dropout
# hyperparameters.LEARNING_RATE = 0.01   
# hyperparameters.DPROB_LIST=[0.5] 

# run 3 - remove dropout, add weight decay
# hyperparameters.WEIGHT_DECAY = 1
# hyperparameters.DPROB_LIST=[0] 

# run 4 - remove dropout, add weight decay
# hyperparameters.WEIGHT_DECAY = 0.1

# run 5 - increase batch size to 256
# hyperparameters.BATCH_SIZE = 256

# run 6 - One cyucle scheduler
# hyperparameters.LEARNING_RATE = 0.001   
# hyperparameters.WEIGHT_DECAY = 10
# hyperparameters.SCHEDULER='OneCyclicLR'
# hyperparameters.SCHEDULER_MAX_LR=0.01
# hyperparameters.SCHEDULER_DIV_FACTOR=25
# hyperparameters.SCHEDULER_FINAL_DIV_FACTOR=1e3
# hyperparameters.EPOCHS = 10

#  run 7 - use pre-trained weights but freeze teh weights - model will 
# hyperparameters.USE_PRE_TRAINED_WEIGHTS = True
# hyperparameters.FREEZE_PRETRAINED = True

# run 8 - Unfreeze the weights
# hyperparameters.FREEZE_PRETRAINED = False

Trainer Configuration

In [24]:
# Fix seed value
Trainer.set_seed()

collate_fn = partial(collate_batch, vocab=stackExchange_vocab)

# Data Loader
train_loader, valid_loader = get_loaders(trainset=se_trainset, validset=se_validset, 
                                         batch_size_=hyperparameters.BATCH_SIZE,
                                         collate_fn=collate_fn)

# cross entropy loss function
loss_function = nn.BCEWithLogitsLoss()

# model
model_stackExchange = MLPCustom(hyperparameters.EMBED_DIM,
                       hyperparameters.VOCAB_SIZE,
                       hyperparameters.HIDDEN_SIZES_LIST,
                       hyperparameters.DPROB_LIST,
                       hyperparameters.OUTPUT_DIM,
                       hyperparameters.NON_LINEARITY,
                       hyperparameters.BATCH_NORM,  )                     
                      #  use_pre_trained_weights =hyperparameters.USE_PRE_TRAINED_WEIGHTS,
                      #  pretrained_weights=pretrained_weights, 
                      #  freeze_pretrained = hyperparameters.FREEZE_PRETRAINED)

def init_weights(m):
  if type(m) == nn.Linear:
      torch.nn.init.kaiming_normal_(m.weight)
      torch.nn.init.zeros_(m.bias)

# apply initialization recursively  to all modules
if hyperparameters.INITIALIZATION =='kaiming':
    model_stackExchange.apply(init_weights)

# OPTIMIZERS
def get_optimizer():
    if hyperparameters.OPTIMIZER == "SGD":
        optimizer = torch.optim.SGD(
            model_stackExchange.parameters(),
            lr=hyperparameters.LEARNING_RATE,
            momentum=hyperparameters.MOMENTUM,
            nesterov=hyperparameters.NESTEROV,
            weight_decay = hyperparameters.WEIGHT_DECAY
        )
    else:
        optimizer = torch.optim.AdamW(
            model_stackExchange.parameters(), 
            lr=hyperparameters.LEARNING_RATE,
            weight_decay = hyperparameters.WEIGHT_DECAY

        )
    return optimizer

optimizer = get_optimizer()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [25]:
model_stackExchange

MLPCustom(
  (non_linearity): ReLU()
  (embedding): EmbeddingBag(90287, 300, mode='mean')
  (module_list): ModuleList(
    (0): Linear(in_features=300, out_features=200, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.0, inplace=False)
    (3): Linear(in_features=200, out_features=10, bias=True)
  )
)

In [26]:
optimizer

AdamW (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.001
    maximize: False
    weight_decay: 0.0
)

Specify Trainer

In [27]:
trainer = Trainer(model=model_stackExchange, optimizer=optimizer,
                  criterion=loss_function, device=device)
# set loaders
trainer.set_loaders(train_loader, valid_loader)

Set Trainer based on Hyperparameters

In [28]:
# Reset Optimizer
trainer.set_optimizer(get_optimizer())

# set metric -- optional
train_metric = MultilabelHammingDistance(num_labels=10)
valid_metric = MultilabelHammingDistance(num_labels=10)
trainer.set_metric(train_metric.to(device), valid_metric.to(device))

# set checkpoint -- OPTIONAL
trainer.set_checkpoint(save_path=run_folder,
                       save_best=True, save_every_n_epochs=5, save_last_epoch=True)

# set early stopping
trainer.set_early_stopping(patience = hyperparameters.EARLY_STOP_PATIENCE)

# set gradient clipping
trainer.set_gradient_clipping(hyperparameters.CLIP_TYPE, hyperparameters.CLIP_VALUE, norm_type=2)

# set scheduler
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(trainer.optimizer, patience=hyperparameters.SCHEDULER_PATIENCE,
#                                           factor=hyperparameters.SCHEDULER_FACTOR)

steps_per_epoch = len(trainer.train_loader)
# scheduler = torch.optim.lr_scheduler.OneCycleLR(trainer.optimizer, 
#                                                 hyperparameters.SCHEDULER_MAX_LR, 
#                                                 steps_per_epoch=steps_per_epoch,
#                                                 epochs=hyperparameters.EPOCHS,
#                                                 div_factor=hyperparameters.SCHEDULER_DIV_FACTOR,
#                                                 final_div_factor=hyperparameters.SCHEDULER_FINAL_DIV_FACTOR)

# trainer.set_lr_scheduler(scheduler=scheduler)

In [29]:
# set wandb -- OPTIONAL
trainer.set_wandb(project_name=project_name,
                  run_name=run_name, config=hyperparameters, log_batch=True, log_frequency=log_frequency)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


wandb: logging graph, to disable use `wandb.watch(log_graph=False)`


Sanity Check

In [30]:
trainer.sanity_check(num_classes=2)

/content/drive/My Drive/NLP/custom-functions/data_preparation_HW7.py:72: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels = torch.tensor(labels, dtype=torch.float64)


Actual loss: 0.6994191809309996
Expected Theoretical loss: 0.6931471805599453


Trainer Model

In [31]:
trainer.train(num_epochs=hyperparameters.EPOCHS, multilabel=True)

Epoch 1/50 - Train Loss: 0.2381, Train Metric: 0.0916, Train Time: 0:00:24.651279
Epoch 1/50 - Val Loss: 0.1516, Val Metric: 0.0554, Val Time: 0:00:06.710697
Current Learning rate is 0.001

Epoch 2/50 - Train Loss: 0.1270, Train Metric: 0.0458, Train Time: 0:00:25.454594
Epoch 2/50 - Val Loss: 0.1234, Val Metric: 0.0448, Val Time: 0:00:06.050181
Current Learning rate is 0.001

Epoch 3/50 - Train Loss: 0.0999, Train Metric: 0.0354, Train Time: 0:00:25.289278
Epoch 3/50 - Val Loss: 0.1115, Val Metric: 0.0400, Val Time: 0:00:06.982883
Current Learning rate is 0.001

Epoch 4/50 - Train Loss: 0.0820, Train Metric: 0.0281, Train Time: 0:00:24.892560
Epoch 4/50 - Val Loss: 0.1050, Val Metric: 0.0375, Val Time: 0:00:06.037698
Current Learning rate is 0.001

Epoch 5/50 - Train Loss: 0.0685, Train Metric: 0.0230, Train Time: 0:00:24.989752
Epoch 5/50 - Val Loss: 0.1023, Val Metric: 0.0356, Val Time: 0:00:06.991926
Current Learning rate is 0.001

Epoch 6/50 - Train Loss: 0.0574, Train Metric: 0.0

KeyboardInterrupt: ignored

In [ ]:
trainer.plot_history()

In [ ]:
import matplotlib.pyplot as plt
plt.plot([lr for list_ in trainer.learning_rates[1:] for lr in list_ ])

Get Accuracy & Predictions

In [ ]:
test_loader = get_test_loaders(se_testset, batch_size_=hyperparameters.BATCH_SIZE,
                               collate_fn=collate_fn)

In [ ]:
trainer.timestamp

In [ ]:
file = 'checkpoint_' + str(trainer.timestamp) + '_best.pt'
trainer.load_checkpoint(run_folder /file)

In [ ]:
# Get the prediction and accuracy for the test dataset
predictions_test, target_test = trainer.predict(
    test_loader, return_targets=True, multilabel=True)
predictions_train, target_train = trainer.predict(
    train_loader, return_targets=True, multilabel=True)
predictions_valid, target_valid = trainer.predict(
    valid_loader, return_targets=True, multilabel=True)

In [ ]:
metric = multilabel_hamming_distance
hd_train = metric(predictions_train, target_train, num_labels=10)
hd_valid = metric(predictions_valid, target_valid, num_labels=10)
hd_test = metric(predictions_test, target_test, num_labels=10)


In [ ]:
# Print Hammo=imng Distance
print('Test hamming distance', hd_test )
print('Train hamming distance', hd_train )
print('Valid hamming distance', hd_valid )

In [ ]:
from torchmetrics.functional.classification import multilabel_accuracy

In [ ]:
metric = multilabel_accuracy
acc_train = metric(predictions_train, target_train, num_labels=10)
acc_valid = metric(predictions_valid, target_valid, num_labels=10)
acc_test = metric(predictions_test, target_test, num_labels=10)


In [ ]:
# Print Hammo=imng Distance
print('Test accuracy', acc_test)
print('Train accuracy', acc_train)
print('Valid accuracy', acc_valid)

In [ ]:
print('Test accuracy', 1- acc_test)
print('Train accuracy', 1- acc_train)
print('Valid accuracy', 1- acc_valid)